In [1]:
import os
import copy
import numpy as np

In [17]:
def explore_interface(target_index, remove_grid, innercity_grid, width):
    latgrd = 2160
    longrd = 4320
    cama_dir = "/mnt/c/Users/tsimk/Downloads/dotfiles/h08/camacity"
    glob_dir = "/mnt/c/Users/tsimk/Downloads/dotfiles/h08/global_city"
    city_path = f"{cama_dir}/dat/cty_lst_/create_text/camacity_third.txt"
    cmsk_dir = f"{cama_dir}/dat/cty_msk_"
    ctymsk_path = f"{cmsk_dir}/city_clrd0000.gl5"
    elvmin_path = f"{glob_dir}/dat/elv_min_/elevtn.CAMA.gl5"
    rivnum_path = f"{glob_dir}/dat/riv_num_/rivnum.CAMA.gl5"
    rivara_path = f"{glob_dir}/dat/riv_ara_/rivara.CAMA.gl5"
    rivnxl_path = f"{glob_dir}/dat/riv_nxl_/rivnxl.CAMA.gl5"

    with open(city_path, "r") as input_file:
        lines = input_file.readlines()

    line = lines[target_index - 1]
    parts = line.split('|')
    parts = [item.strip() for item in parts]
    city_num = int(parts[0])
    ovlp_state = parts[1]
    clst_state = parts[2]
    cnt_lat = float(parts[9])
    cnt_lon = float(parts[10])

    lonmin = float(cnt_lon - width)
    lonmax = float(cnt_lon + width)
    latmin = float(cnt_lat - width)
    latmax = float(cnt_lat + width)

    lat_length = np.linspace(-90, 90, latgrd+1)
    lon_length = np.linspace(-180, 180, longrd+1)
    lat_start, lat_end = np.searchsorted(lat_length, [latmin, latmax])
    lon_start, lon_end = np.searchsorted(lon_length, [lonmin, lonmax])

    if lat_start%3 == 0:
        lat_start = lat_start
    elif lat_start%3 == 1:
        lat_start -= 1
    elif lat_start%3 == 2:
        lat_start += 1
    if lon_start%3 == 0:
        lon_start = lon_start
    elif lon_start%3 == 1:
        lon_start -= 1
    elif lon_start%3 == 2:
        lon_start += 1

    width_grid = width * 12 * 2
    lat_end = lat_start + width_grid
    lon_end = lon_start + width_grid

    if clst_state == 'NoMK' or ovlp_state == 'RMVD':
        return

    g_mask_original = np.fromfile(ctymsk_path, 'float32').reshape(latgrd, longrd)
    g_mask = np.where(g_mask_original == city_num, 1, 0)
    g_mask = np.flipud(g_mask_original)
    g_mask_cropped = g_mask[lat_start:lat_end, lon_start:lon_end]
    g_mask_cropped = np.flipud(g_mask_cropped)

    grid_count = int(float(parts[7]))
    if grid_count == 1:
        return 

    g_elv = np.fromfile(elvmin_path, 'float32').reshape(latgrd, longrd)
    g_elv = np.ma.masked_where(g_elv >= 1E20, g_elv)
    g_elv = np.flipud(g_elv)
    g_elv_cropped = g_elv[lat_start:lat_end, lon_start:lon_end]
    g_elv_cropped = np.flipud(g_elv_cropped)

    g_rivnum = np.fromfile(rivnum_path, 'float32').reshape(latgrd, longrd)
    g_rivnum = np.ma.masked_where(g_rivnum >= 1E20, g_rivnum)
    g_rivnum = np.flipud(g_rivnum)
    g_rivnum_cropped = g_rivnum[lat_start:lat_end, lon_start:lon_end]
    g_rivnum_cropped = np.flipud(g_rivnum_cropped)
    g_rivnum_cropped = np.ma.masked_where(~np.isfinite(g_rivnum_cropped) | (g_rivnum_cropped == 0), g_rivnum_cropped)

    g_rivara = np.fromfile(rivara_path, 'float32').reshape(latgrd, longrd)
    g_rivara = np.ma.masked_where(g_rivara >= 1E20, g_rivara)
    g_rivara = np.flipud(g_rivara)
    g_rivara_cropped = g_rivara[lat_start:lat_end, lon_start:lon_end]
    g_rivara_cropped = np.flipud(g_rivara_cropped)
    g_rivara_cropped = np.ma.masked_where(~np.isfinite(g_rivara_cropped) | (g_rivara_cropped == 0), g_rivara_cropped)

    g_rivnxl = np.fromfile(rivnxl_path, 'float32').reshape(latgrd, longrd)
    g_rivnxl = np.flipud(g_rivnxl)
    g_rivnxl = np.ma.masked_where(g_rivnxl >= 1E20, g_rivnxl)
    g_rivnxl_cropped = g_rivnxl[lat_start:lat_end, lon_start:lon_end]
    g_rivnxl_cropped = np.flipud(g_rivnxl_cropped)
    g_rivnxl_cropped = np.ma.masked_where(~np.isfinite(g_rivnxl_cropped) | (g_rivnxl_cropped == 0), g_rivnxl_cropped)

    g_rivnum_cropped_city = np.where(g_mask_cropped == city_num, g_rivnum_cropped, np.nan)
    g_rivnum_cropped_city = np.ma.masked_where(~np.isfinite(g_rivnum_cropped_city) | (g_rivnum_cropped_city == 0), g_rivnum_cropped_city)

    dtype = [('rivara', 'float32'), ('rivnum', 'float32')]
    g_ara_num_cropped = np.empty(g_rivara_cropped.shape, dtype=dtype)

    g_ara_num_cropped['rivara'] = g_rivara_cropped
    g_ara_num_cropped['rivnum'] = g_rivnum_cropped

    g_rivnum_cropped_masked = np.ma.masked_array(g_rivnum_cropped, np.isnan(g_rivnum_cropped))
    unique_values_org, counts_org = np.unique(g_rivnum_cropped_masked.compressed(), return_counts=True)
    value_counts_dict = dict(zip(unique_values_org, counts_org))
    sorted_dict_by_value_descending = dict(sorted(value_counts_dict.items(), key=lambda item: item[1], reverse=True))
    filtered_dict_A = {key: value for key, value in sorted_dict_by_value_descending.items() if value >= remove_grid}
    Rivnum_A_array = np.ma.masked_all(g_rivnum_cropped_masked.shape, dtype='float32')
    for rivnum_id in filtered_dict_A.keys():
        matching_positions = np.where(g_rivnum_cropped_masked.data == rivnum_id)
        Rivnum_A_array[matching_positions] = rivnum_id
    Rivnum_A_array = np.ma.masked_where(~np.isfinite(Rivnum_A_array) | (Rivnum_A_array == 0), Rivnum_A_array)

    invalid_mask = np.isnan(Rivnum_A_array) | (Rivnum_A_array == 0)
    Rivnum_A_array_citymasked = np.ma.masked_where((g_mask_cropped != city_num) | invalid_mask, Rivnum_A_array)
    unique_values_A, counts_A = np.unique(Rivnum_A_array_citymasked.compressed(), return_counts=True)
    value_counts_dict_A = dict(zip(unique_values_A, counts_A))

    rivara_max_array_A = np.ma.masked_all(g_ara_num_cropped.shape, dtype='float32')
    for rivnum_id in value_counts_dict_A.keys():
        matching_positions = np.where(Rivnum_A_array_citymasked == rivnum_id)
        max_rivara_position = np.argmax(g_rivara_cropped[matching_positions])
        rivara_max_array_A[matching_positions[0][max_rivara_position], matching_positions[1][max_rivara_position]] = rivnum_id

    vfunc = np.vectorize(l_coordinate_to_tuple, otypes=[tuple])
    riv_nxlonlat = np.empty(g_rivnxl_cropped.shape, dtype=tuple)
    mask = ~np.isnan(g_rivnxl_cropped)
    riv_nxlonlat[mask] = vfunc(g_rivnxl_cropped[mask])
    riv_nxlonlat_shape = (riv_nxlonlat.shape[0], riv_nxlonlat.shape[1], 2)
    riv_nxlonlat_lst = []
    for row in riv_nxlonlat:
        for x, y in row:
            modified_x = width_grid - (x - lat_start)
            modified_y = y - lon_start
            riv_nxlonlat_lst.append((modified_x, modified_y))
    riv_nxlonlat_cropped = np.array(riv_nxlonlat_lst).reshape(riv_nxlonlat_shape)
    riv_nxlonlat_cropped = riv_nxlonlat_cropped.astype(int)

    path_dict = {}
    riv_path_array_A = np.ma.masked_all(rivara_max_array_A.shape, dtype='float32')
    visited_coords = set()
    for uid in unique_values_A:
        coords_a = np.argwhere(rivara_max_array_A == uid)
        riv_path_array_A[coords_a[0][0], coords_a[0][1]] = uid
        if coords_a.size > 0:
            target_coord = tuple(coords_a[0]) 
            path_coords = [target_coord]
            for _ in range(len(g_mask_cropped)):
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                matched_coords = np.argwhere(np.all(target_coord == riv_nxlonlat_cropped, axis=2))
                if len(matched_coords) == 0:
                    break
                unvisited_matched = [tuple(coord) for coord in matched_coords if tuple(coord) not in visited_coords]
                if not unvisited_matched:
                    break
                rivara_values = [g_rivara_cropped[coord[0], coord[1]] for coord in unvisited_matched]
                max_index = np.argmax(rivara_values)
                best_coord = unvisited_matched[max_index]
                riv_path_array_A[best_coord[0], best_coord[1]] = uid
                target_coord = best_coord 
                path_coords.append(target_coord)
            path_dict[uid] = path_coords

    fill_value = 1e20
    riv_path_array_filled = riv_path_array_A.filled(fill_value)
    riv_path_city_A = np.where(g_mask_cropped==city_num, riv_path_array_filled, fill_value)
    riv_path_city_B = copy.deepcopy(riv_path_city_A)
    for uid in unique_values_A:
        count = 0
        mask = (riv_path_city_A == uid)
        count = np.sum(mask)
        if count < innercity_grid:
            riv_path_city_B[riv_path_city_B== uid] = fill_value
    riv_path_city_B = np.ma.masked_where(riv_path_city_B >= fill_value, riv_path_city_B)

    unique_values_B, _ = np.unique(riv_path_city_B.compressed(), return_counts=True)

    Rivnum_B_array = np.ma.masked_all(g_rivnum_cropped_masked.shape, dtype='float32')
    for uid in unique_values_B:
        row_indices, col_indices = np.where(Rivnum_A_array == uid)
        Rivnum_B_array[row_indices, col_indices] = uid
    
    rivara_max_array_B = np.ma.masked_all(g_ara_num_cropped.shape, dtype='float32')
    for rivnum_id in unique_values_B:
        matching_positions = np.where(Rivnum_A_array_citymasked == rivnum_id)
        max_rivara_position = np.argmax(g_rivara_cropped[matching_positions])
        rivara_max_array_B[matching_positions[0][max_rivara_position], matching_positions[1][max_rivara_position]] = rivnum_id

    path_dict = {}
    riv_path_array_B = np.ma.masked_all(rivara_max_array_B.shape, dtype='float32')
    visited_coords = set()
    for uid in unique_values_B:
        coords_a = np.argwhere(rivara_max_array_B == uid)
        riv_path_array_B[coords_a[0][0], coords_a[0][1]] = uid
        if coords_a.size > 0:
            target_coord = tuple(coords_a[0]) 
            path_coords = [target_coord]
            for _ in range(len(g_mask_cropped)):
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                matched_coords = np.argwhere(np.all(target_coord == riv_nxlonlat_cropped, axis=2))
                if len(matched_coords) == 0:
                    break
                unvisited_matched = [tuple(coord) for coord in matched_coords if tuple(coord) not in visited_coords]
                if not unvisited_matched:
                    break
                rivara_values = [g_rivara_cropped[coord[0], coord[1]] for coord in unvisited_matched]
                max_index = np.argmax(rivara_values)
                best_coord = unvisited_matched[max_index]
                riv_path_array_B[best_coord[0], best_coord[1]] = uid
                target_coord = best_coord 
                path_coords.append(target_coord)
            path_dict[uid] = path_coords
    
    josui_lst = []
    for key_num in unique_values_B:
        indices = np.argwhere(riv_path_city_B == key_num)
        rivara_values = [g_rivara_cropped[coord[0], coord[1]] for coord in indices]
        min_arg = np.argmin(rivara_values)
        josui = indices[min_arg]
        josui_lst.append(josui)

    josui_array = np.ma.masked_all(rivara_max_array_B.shape, dtype='float32')
    for matching_position, uid in zip(josui_lst, unique_values_B):
        josui_array[matching_position[0], matching_position[1]] = uid
    
    gesui_array = copy.deepcopy(rivara_max_array_B)
    
    no_prf_flag = False
    josui_array = np.ma.filled(josui_array, fill_value=0)
    prf_coords = np.where(josui_array>0)
    if len(prf_coords[0]) == 0:
        print(f"no purification")
        no_prf_flag = True
        josui_array, gesui_array = nomainriver_option(g_mask_cropped, 
                                                      city_num,
                                                      g_rivnum_cropped_city, 
                                                      g_elv_cropped, 
                                                      g_rivara_cropped,
                                                      )
        print(f"non_prf -> tentative prf")

    josui_for_save = np.ma.masked_all(g_rivara.shape, dtype='float32')
    josui_for_save[lat_start:lat_end, lon_start:lon_end] = np.flipud(josui_array)
    josui_for_save = np.ma.filled(josui_for_save, fill_value=0)
    josui_for_save = np.where(josui_for_save > 0, city_num, josui_for_save)
    josui_for_save = np.flipud(josui_for_save)

    gesui_for_save = np.ma.masked_all(g_rivara.shape, dtype='float32')
    gesui_for_save[lat_start:lat_end, lon_start:lon_end] = np.flipud(gesui_array)
    gesui_for_save = np.ma.filled(gesui_for_save, fill_value=0)
    gesui_for_save = np.where(gesui_for_save > 0, city_num, gesui_for_save)
    gesui_for_save = np.flipud(gesui_for_save)
    return josui_for_save, gesui_for_save

def l_coordinate_to_tuple(lcoordinate, a=2160, b=4320):
    lat_l = a - ((lcoordinate - 1) // b)
    lon_l = (lcoordinate) % b - 1
    return (lat_l, lon_l)

def nomainriver_option(citymask, city_num, rivnum, elevation, rivara):
    unique_values, counts = np.unique(rivnum.compressed(), return_counts=True)
    uid_dict = dict(zip(unique_values, counts))
    max_key = max(uid_dict, key=uid_dict.get)
    if max_key > 1:
        elv_indices = np.argwhere(rivnum == max_key)
        elv_values = [elevation[coord[0], coord[1]] for coord in elv_indices]
        elv_maxarg = np.argmax(elv_values)
        josui_coord = elv_indices[elv_maxarg]
        josui_array = np.zeros(rivnum.shape, dtype='float32')
        josui_array[josui_coord[0], josui_coord[1]] = max_key
        ara_indices = np.argwhere((citymask == city_num) & (josui_array != rivnum[josui_coord[0], josui_coord[1]]))
        ara_values = [rivara[coord[0], coord[1]] for coord in ara_indices]
        if ara_values:
            ara_argmax = np.argmax(ara_values)
            gesui_coord = ara_indices[ara_argmax]
        else:
            print(f"ara_indices is empty -> argmin_elv for gesui")
            elv_minarg = np.argmax(elv_values)
            gesui_coord = elv_indices[elv_minarg]
    else:
        elv_indices = np.argwhere(citymask == city_num)
        elv_values = [elevation[coord[0], coord[1]] for coord in elv_indices]
        elv_maxarg = np.argmax(elv_values)
        josui_coord = elv_indices[elv_maxarg]
        josui_array = np.zeros(rivnum.shape, dtype='float32')
        josui_array[josui_coord[0], josui_coord[1]] = rivnum[josui_coord[0], josui_coord[1]]
        ara_indices = np.argwhere((citymask == city_num) & (josui_array != rivnum[josui_coord[0], josui_coord[1]]))
        ara_values = [rivara[coord[0], coord[1]] for coord in ara_indices]
        if ara_values:
            ara_argmax = np.argmax(ara_values)
            gesui_coord = ara_indices[ara_argmax]
        else:
            print(f"ara_indices is empty -> argmin_elv for gesui")
            elv_minarg = np.argmax(elv_values)
            gesui_coord = elv_indices[elv_minarg]
    josui_array = np.ma.masked_all(rivnum.shape, dtype='float32')
    josui_array[josui_coord[0], josui_coord[1]] = rivnum[josui_coord[0], josui_coord[1]]
    gesui_array = np.ma.masked_all(rivnum.shape, dtype='float32')
    gesui_array[gesui_coord[0], gesui_coord[1]] = rivnum[gesui_coord[0], gesui_coord[1]]
    return josui_array, gesui_array

In [19]:
josui_array, gesui_array = explore_interface(target_index=1429, remove_grid=5, innercity_grid=2, width=1)
print(josui_array.shape)

no purification
non_prf -> tentative prf
(2160, 4320)


In [ ]:
def get_updown_rivpath(city_num, prf, swg, upperindex, lowerindex, leftindex, rightindex, riv_num, riv_nxlonlat, riv_ara, riv_out, ibt=False):
    rivpath_dict = {}
    coord_list = []
    prf_cropped = prf[int(upperindex):int(lowerindex), int(leftindex):int(rightindex)]
    coord_list.append(np.where(prf_cropped == 1))

    # downstream exploration
    count = 0
    for srcd in coord_list:
        yyy = srcd[0]
        xxx = srcd[1]
        for y, x in zip(yyy, xxx):
            target_coord = (y, x) 
            path_coords = [target_coord]
            visited_coords = set()
            while True:
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                target_row, target_col = target_coord
                next_coord = riv_nxlonlat[target_row, target_col]
                if next_coord.size == 0 or next_coord.shape != (2,):
                    break
                target_coord = (next_coord[0], next_coord[1])
                path_coords.append(target_coord)
            rivpath_dict[count] = path_coords
            count += 1

    # upstream exploration
    for srcd in coord_list:
        yyy = srcd[0]
        xxx = srcd[1]
        for y, x in zip(yyy, xxx):
            target_coord = (y, x)
            path_coords = [target_coord]
            visited_coords = set()
            while True:
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                matched_coords = np.argwhere(np.all(target_coord == riv_nxlonlat, axis=2))
                if len(matched_coords) == 0:
                    break
                unvisited_matched = [tuple(coord) for coord in matched_coords if tuple(coord) not in visited_coords]
                if not unvisited_matched:
                    break
                rivara_values = [riv_ara[coord[0], coord[1]] for coord in unvisited_matched]
                max_index = np.argmax(rivara_values)
                best_coord = unvisited_matched[max_index]
                target_coord = best_coord
                path_coords.append(target_coord)
            rivpath_dict[count] = path_coords
            count += 1
    return rivpath_dict

In [ ]:
def plot_basin_path_foreback(extent, citymask, data_fore, data_back, Rivnum_masked_array, results_dict_fore=None, results_dict_back=None, gs_position=None, fig=None):
    unique_fore = np.unique(data_fore)
    filtered_fore = [i for i in unique_fore if i != 0]
    color_code_fore = ['#4169e1', '#fffacd', '#c71585', '#00fa9a', '#ba55d3', '#48d1cc', '#ffc0cb', '#ffa07a']
    
    color_masked_fore = np.ma.masked_where(~np.isin(Rivnum_masked_array, filtered_fore), Rivnum_masked_array)
    color_filled_fore = color_masked_fore.filled(-1)
    num_colors_fore, index_dict_fore, cmap_fore = pulp_cmap(color_filled_fore, color_code_fore)
    
    color_code_back = ['#4169e1', '#fffacd', '#c71585', '#00fa9a', '#ba55d3', '#48d1cc', '#ffc0cb', '#ffa07a']
    color_masked_back = np.ma.masked_where(Rivnum_masked_array >= 1e20, Rivnum_masked_array)
    color_filled_back = color_masked_back.filled(-1)
    num_colors_back, index_dict_back, cmap_back = pulp_cmap(color_filled_back, color_code_back)

    # data
    data_masked_fore = np.ma.masked_where(data_fore >= 1e20, data_fore)
    data_filled_fore = data_masked_fore.filled(-1)
    indexed_data_fore = np.vectorize(get_index)(data_filled_fore, index_dict_fore)
    indexed_data_masked_fore = np.ma.masked_where(indexed_data_fore<0, indexed_data_fore)
    
    data_masked_back = np.ma.masked_where(data_back >= 1e20, data_back)
    data_filled_back = data_masked_back.filled(-1)
    indexed_data_back = np.vectorize(get_index)(data_filled_back, index_dict_back)
    indexed_data_masked_back = np.ma.masked_where(indexed_data_back<0, indexed_data_back)
    
    # extent
    projection = ccrs.PlateCarree()

    if gs_position is not None:
        ax = fig.add_subplot(gs_position, projection=projection)
    else:
        fig = plt.figure(figsize=(5,5))
        ax = plt.subplot(projection=projection)
        
    ax.set_extent(extent, projection)
    #gl = ax.gridlines(draw_labels=True, zorder=-1)
    #gl.top_labels = False   
    #gl.bottom_labels = False   
    #gl.right_labels = False 
    #gl.left_labels = False 
    ax.add_feature(cfea.OCEAN, color='#BBBBBB')
    ax.add_feature(cfea.LAKES, color='#BBBBBB')
    ax.coastlines(zorder=0)
    
    rect = plt.Rectangle(
        (0, 0), 1, 1, transform=ax.transAxes,
        linewidth=10, edgecolor='black', facecolor='none', clip_on=False)
    ax.add_patch(rect)
    
    img_back = ax.imshow(indexed_data_masked_back,
                    origin='upper',
                    extent=extent,
                    transform=projection,
                    cmap=cmap_back,
                    vmin=-0.5, vmax=num_colors_back-0.5,
                    alpha=0.1)
    
    img_fore = ax.imshow(indexed_data_masked_fore,
                    origin='upper',
                    extent=extent,
                    transform=projection,
                    cmap=cmap_fore,
                    vmin=-0.5, vmax=num_colors_fore-0.5,
                    alpha=1.0)
    
    if results_dict_fore is not None:
        width = 1
        for key, path in results_dict_fore.items():
            lat_coords, lon_coords = zip(*path)
            lat_coords = [extent[3] - ((lat + (1/(width * 2)))*(1/12)) for lat in lat_coords]
            lon_coords = [(lon + (1/(width * 2)))*(1/12) + extent[0] for lon in lon_coords]
            ax.plot(lon_coords, lat_coords, marker='o', markersize=1.5, color='black', linestyle='-', linewidth=1, transform=projection, zorder=2)
            
    stw = draw_mask_sotowaku(extent, citymask, sotowaku_color='white', ax=ax) 

In [ ]:
west, east, south, north = 72, 83, 13, 21

In [25]:
import os
import numpy as np
import json
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
import matplotlib.ticker as mticker

def geography(left, right, bottom, top):
    resolution = 12
    upperindex = (90-top)*resolution
    lowerindex = (90-bottom)*resolution
    leftindex = (180+left)*resolution
    rightindex = (180+right)*resolution
    rgnshape = (lowerindex-upperindex, rightindex-leftindex)
    img_extent = (left, right, bottom, top)
    return upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent
    
def l_coordinate_to_tuple(lcoordinate, a=2160, b=4320):
    lat_l = ((lcoordinate - 1) // b)
    lon_l = (lcoordinate) % b - 1
    return (lat_l, lon_l)

def nxtl2nxtxy(rgnfile, upperindex, leftindex):
    vfunc = np.vectorize(l_coordinate_to_tuple, otypes=[tuple])
    riv_nxtxy = np.empty(rgnfile.shape, dtype=tuple)
    mask = ~np.isnan(rgnfile)
    riv_nxtxy[mask] = vfunc(rgnfile[mask])
    riv_nxtxy_shape = (riv_nxtxy.shape[0], riv_nxtxy.shape[1], 2)
    riv_nxtxy_lst = []
    for row in riv_nxtxy:
        for y, x in row:
            modified_y = y - upperindex
            modified_x = x - leftindex
            riv_nxtxy_lst.append((modified_y, modified_x))
    riv_nxtxy_cropped = np.array(riv_nxtxy_lst).reshape(riv_nxtxy_shape)
    riv_nxtxy_cropped = riv_nxtxy_cropped.astype(int)
    return riv_nxtxy_cropped

def grid_to_latlon(row, col, extent):
    delta = 360/4320
    x_left, y_top = extent[0], extent[3]
    lon = x_left + (col+0.5) * delta
    lat = y_top - (row+0.5) * delta
    return lon, lat
    
def draw_mask_sotowaku(extent, mask_cropped, ax=None, sotowaku_color='#dc143c', linewidth=3):
    projection = ccrs.PlateCarree()
    delta = 360/4320
    x_left = extent[0]
    y_top = extent[3]
    upper_indices = []
    lower_indices = []
    left_indices = []
    right_indices = []
    rows, cols = mask_cropped.shape
    for i in range(rows-1):
        for j in range(cols-1):
            if mask_cropped[i, j] != 0:
                if mask_cropped[i-1, j] == 0:
                    upper_indices.append((i, j))
                if mask_cropped[i+1, j] == 0:
                    lower_indices.append((i, j))
                if mask_cropped[i, j-1] == 0:
                    left_indices.append((i, j))
                if mask_cropped[i, j+1] == 0:
                    right_indices.append((i, j))
    for up in upper_indices:
        x_index = x_left + delta*(up[1])
        y_index = y_top - delta*(up[0])
        ax.plot([x_index, x_index+delta], [y_index, y_index], color=sotowaku_color, transform=projection, linewidth=linewidth, zorder=1) # top
    for lo in lower_indices:
        x_index = x_left + delta*(lo[1])
        y_index = y_top - delta*(lo[0]+1)
        ax.plot([x_index, x_index+delta], [y_index, y_index], color=sotowaku_color, transform=projection, linewidth=linewidth, zorder=1) # bottom
    for le in left_indices:
        x_index = x_left + delta*(le[1])
        y_index = y_top - delta*(le[0]+1)
        ax.plot([x_index, x_index], [y_index, y_index+delta], color=sotowaku_color, transform=projection, linewidth=linewidth, zorder=1) # left
    for ri in right_indices:
        x_index = x_left + delta*(ri[1]+1)
        y_index = y_top - delta*(ri[0]+1)
        ax.plot([x_index, x_index], [y_index, y_index+delta], color=sotowaku_color, transform=projection, linewidth=linewidth, zorder=1) # right
    return ax

def get_rivpath_dict(riv_num, city_num, upperindex, lowerindex, leftindex, rightindex):
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    prf_dir = f'{h08dir}/camacity/dat/cty_prf_'
    prf_path = f'{prf_dir}/prf_clrd0000.gl5'
    prf = np.fromfile(prf_path, dtype='float32').reshape(2160, 4320)
    swg_dir = f'{h08dir}/camacity/dat/cty_swg_'
    swg_path = f'{swg_dir}/swg_clrd0000.gl5'
    swg = np.fromfile(swg_path, dtype='float32').reshape(2160, 4320)
    int_dir = f'{h08dir}/global_city/dat/cty_int_/100km_samebasin'
    rivnxl_path = f"{h08dir}/global_city/dat/riv_nxl_/rivnxl.CAMA.gl5"
    rivnxl = np.fromfile(rivnxl_path, 'float32').reshape(2160, 4320)
    rivnxl_cropped = rivnxl[upperindex:lowerindex, leftindex:rightindex]
    riv_nxlonlat_cropped = nxtl2nxtxy(rivnxl_cropped, upperindex, leftindex)
    rivara_path = f'{h08dir}/global_city/dat/riv_ara_/rivara.CAMA.gl5'
    rivara = np.fromfile(rivara_path, 'float32').reshape(2160, 4320)
    rivara_cropped = rivara[upperindex:lowerindex, leftindex:rightindex]
    rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
    rivnum = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320) 
    rivnum = rivnum.astype(int)
    rivnum_cropped = rivnum[upperindex:lowerindex, leftindex:rightindex]
    rivout_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
    rivout = np.fromfile(rivout_path, dtype='float32').reshape(2160, 4320)
    rivout_cropped = rivout[upperindex:lowerindex, leftindex:rightindex]

    rivpath_dict = {}
    city_num_dict = {}
    coord_list = []
    prf_masked = np.where(prf==city_num, 1, 0)
    swg_masked= np.where(swg==city_num, 1, 0)
    int_path = f'{int_dir}/city_{city_num:08}.gl5'
    if np.sum(prf_masked) < 1:
        print(f'{city_num} is invalid prf')
    else:
        prf_cropped = prf_masked[upperindex:lowerindex, leftindex:rightindex]
        swg_cropped = swg_masked[upperindex:lowerindex, leftindex:rightindex]
        intake = np.fromfile(int_path, dtype='float32').reshape(2160, 4320)
        intake_cropped = intake[upperindex:lowerindex, leftindex:rightindex]
        coord_list.append(np.where(prf_cropped == 1))
        coord_list.append(np.where(swg_cropped == 1))
        coord_list.append(np.where(intake_cropped == 1))

    # downstream exploration
    count = 0
    for srcd in coord_list:
        yyy = srcd[0]
        xxx = srcd[1]
        for y, x in zip(yyy, xxx):
            target_coord = (y, x) 
            path_coords = [target_coord]
            visited_coords = set()
            while True:
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                target_row, target_col = target_coord
                next_coord = riv_nxlonlat_cropped[target_row, target_col]
                if next_coord.size == 0 or next_coord.shape != (2,):
                    break
                target_coord = (next_coord[0], next_coord[1])
                path_coords.append(target_coord)
            city_num_dict[count] = path_coords
            count += 1

    # upstream exploration
    for srcd in coord_list:
        yyy = srcd[0]
        xxx = srcd[1]
        for y, x in zip(yyy, xxx):
            target_coord = (y, x)
            path_coords = [target_coord]
            visited_coords = set()
            while True:
                if target_coord in visited_coords:
                    break
                visited_coords.add(target_coord)
                matched_coords = np.argwhere(
                    np.all(target_coord == riv_nxlonlat_cropped, axis=2)
                )
                if len(matched_coords) == 0:
                    break
                unvisited_matched = [
                    tuple(coord) for coord in matched_coords if tuple(coord) not in visited_coords
                ]
                if not unvisited_matched:
                    break
                rivara_values = [
                    rivara_cropped[coord[0], coord[1]] for coord in unvisited_matched
                ]
                max_index = np.argmax(rivara_values)
                best_coord = unvisited_matched[max_index]
                target_coord = best_coord
                path_coords.append(target_coord)
            city_num_dict[count] = path_coords
            count += 1
    rivpath_dict[city_num] = city_num_dict
    return rivpath_dict

def draw_interface(city_num, prf, swg):
    riv_num = 64
    #
    left = 70
    right = 85
    bottom = 10
    top = 25
    #
    resolution=12
    nx, ny = (right - left)*resolution, (top - bottom)*resolution
    rgnshape = (ny, nx)
    img_extent = (left, right, bottom, top)
    extent = (-180, 180, -90, 90)
    upperindex = (90-top)*resolution
    lowerindex = (90-bottom)*resolution
    leftindex = (180+left)*resolution
    rightindex = (180+right)*resolution
    
###########################################################################################
    h08dir = '/mnt/c/Users/tsimk/Downloads/dotfiles/h08'
    upperindex, lowerindex, leftindex, rightindex, rgnshape, img_extent = geography(left, right, bottom, top)

    # riv
    rivnum_path = f'{h08dir}/global_city/dat/riv_num_/rivnum.CAMA.gl5'
    rivnum = np.fromfile(rivnum_path, dtype='float32').reshape(2160, 4320) 
    rivnum_cropped = rivnum[upperindex:lowerindex, leftindex:rightindex]
    rivnum_masked = np.ma.masked_where(rivnum_cropped != riv_num, rivnum_cropped)
    
    # discharge
    rivout_path = f'{h08dir}/wsi/dat/riv_out_/W5E5LR__00000000.gl5'
    rivout = np.fromfile(rivout_path, dtype='float32').reshape(2160, 4320)
    rivout_cropped = rivout[upperindex:lowerindex, leftindex:rightindex] * 60 * 60 * 24 * 365 / 1000

    # rivnxl
    path_dict = get_rivpath_dict(riv_num, city_num, upperindex, lowerindex, leftindex, rightindex)

    # cmap
    bounds = [0, 1000]
    cmap = mcolors.ListedColormap(['blue', 'blue'])
    norm = mcolors.BoundaryNorm(bounds, cmap.N)
    
    rivnum_cmap = mcolors.ListedColormap(['#edb69b'])
    srivnum_cmap = mcolors.ListedColormap(['#f1d89c'])
    river_cmap = mcolors.ListedColormap(['#57acff'])

    # draw
    projection = ccrs.PlateCarree()
    fig, ax = plt.subplots(figsize=(20,10), subplot_kw={'projection': projection})
    ax.set_extent([left, right, bottom, top], projection)

    # rivnxl
    width = 1
    path_city_dict = path_dict[city_num]
    for key, path_coords in path_city_dict.items():
        path_coords_lat = [coord[0] for coord in path_coords]
        path_coords_lon = [coord[1] for coord in path_coords]
        path_coords_pairs = list(zip(path_coords_lat, path_coords_lon))
        path_coords_grouped = [(path_coords_lat[i:i + 2], path_coords_lon[i:i + 2]) for i in range(len(path_coords) - 1)]
        for lat_coords, lon_coords in path_coords_grouped:
            discharge = rivout_cropped[lat_coords[0], lon_coords[0]]
            if discharge > 1e10:
                linewidth = 2.0
            elif 1e10 > discharge > 1e9:
                linewidth = 1.2
            else:
                linewidth = 0.4
            lat_coords = [img_extent[3] - ((lat + (1/(width * 2)))*(1/12)) for lat in lat_coords]
            lon_coords = [(lon + (1/(width * 2)))*(1/12) + img_extent[0] for lon in lon_coords]
            ax.plot(lon_coords, lat_coords, 
                    markersize=0, linewidth=linewidth, color='#57acff',
                    marker='o', linestyle='-', transform=projection, zorder=1
                   )

    # rivnum
    bg = ax.imshow(rivnum_masked, 
                   alpha=1, cmap=rivnum_cmap, transform=projection, extent=img_extent
                  )

    for srn in sub_rivnum_list:
        srivnum_masked = np.ma.masked_where(rivnum_cropped != srn, rivnum_cropped)
        ax.imshow(srivnum_masked, 
                  alpha=1, cmap=srivnum_cmap, transform=projection, extent=img_extent
                  )

    # city outer
    ctymsk_path = f'{msk_dir}/city_{city_num:08}.gl5'
    citymask = np.fromfile(ctymsk_path, dtype='float32').reshape(2160, 4320)
    citymask_cropped = citymask[upperindex:lowerindex, leftindex:rightindex]
    draw_mask_sotowaku(img_extent, citymask_cropped, ax=ax, sotowaku_color='white', linewidth=1.5)
        
    # prf
    pprf_latlon = [grid_to_latlon(coord[0][0], coord[1][0], img_extent) for _, coord in pprf_dct.items()]
    pprf_lon, pprf_lat = zip(*pprf_latlon)
    ax.scatter(pprf_lon, pprf_lat, s=3, facecolors='none', linewidths=1.1,
            color='blue', transform=ccrs.PlateCarree(), zorder=3)

    # swg
    pswg_latlon = [grid_to_latlon(coord[0][0], coord[1][0], img_extent) for _, coord in pswg_dct.items()]
    pswg_lon, pswg_lat = zip(*pswg_latlon)
    ax.scatter(pswg_lon, pswg_lat, s=3, facecolors='none', linewidths=1.1,
            color='red', transform=ccrs.PlateCarree(), zorder=3)

    # secondary prf
    sprf_latlon = [
        [grid_to_latlon(c[0], c[1], img_extent) for c in zip(coord[0], coord[1])] if coord is not None else [(None, None)] 
        for n, coord in sprf_dct.items()
    ]
    
    for coord_list in sprf_latlon:
        for lon, lat in coord_list:
            if lon is not None and lat is not None:
                ax.plot(lon, lat, marker='x', color='blue', markersize=3, 
                        transform=ccrs.PlateCarree(), zorder=2)
                
    # secondary swg
    sswg_latlon = [
        [grid_to_latlon(c[0], c[1], img_extent) for c in zip(coord[0], coord[1])] if coord is not None else [(None, None)] 
        for n, coord in sswg_dct.items()
    ]
    
    for coord_list in sswg_latlon:
        for lon, lat in coord_list:
            if lon is not None and lat is not None:
                ax.plot(lon, lat, marker='x', color='red', markersize=3, 
                        transform=ccrs.PlateCarree(), zorder=2)

    ax.add_feature(cfeature.LAND, edgecolor='none', facecolor='#eaeaea', zorder=-3)
    ax.add_feature(cfeature.OCEAN, edgecolor='none', facecolor='#ffffff')
    ax.add_feature(cfeature.LAKES, edgecolor='none', facecolor='#ffffff', zorder=-1)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.3,edgecolor='black')
    ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=1.5, edgecolor='#d0cfd4', zorder=-3)
    ax.gridlines(visible=False)

    west, east, south, north = 72, 83, 13, 21
    ax.set_xlim(west, east)
    ax.set_ylim(south, north)
    
    gl = ax.gridlines(draw_labels=True, zorder=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlocator = mticker.FixedLocator(np.arange(round(west/10)*10, east, 5))
    gl.ylocator = mticker.FixedLocator(np.arange(round(south/10)*10, north, 5))
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True
    gl.xlines = False
    gl.ylines = False
    gl.xlabel_style = {'size': 20, 'color': 'black'}
    gl.ylabel_style = {'size': 20, 'color': 'black'}
    
    ax.gridlines(visible=False)
    ax.spines['geo'].set_visible(True)
    
    plt.show()

SyntaxError: expected ':' (2161116525.py, line 265)

In [26]:
draw_interface(1429, josui_array, gesui_array)

UnboundLocalError: local variable 'sint_coord' referenced before assignment